In [1]:
from __future__ import division  
import nltk, re, pprint
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from collections import Counter

import gensim
from gensim import corpora, models

import pyLDAvis
import pyLDAvis.gensim

#nltk.download('punkt')
import numpy as np
import pandas as pd

In [2]:
from operator import itemgetter
import heapq
import collections
def least_common_values(array, to_find=None):
    counter = collections.Counter(array)
    if to_find is None:
        return sorted(counter.items(), key=itemgetter(1), reverse=False)
    return heapq.nsmallest(to_find, counter.items(), key=itemgetter(1))

In [3]:
#Want a list of the top N words in the entire corpus, so that the matrix we're working with isn't insane
f = open('../Papers/00/all.txt')
raw = f.read()
tokens = word_tokenize(raw.decode('utf-8'))
tokens = [w.lower() for w in tokens]
words = [word for word in tokens if word.isalpha()]
stop_words = set(stopwords.words('english'))
ignore_words = ['annual','review','journal','may','anderson','reviews','ann','rev','rights','reserved','institution',
               'annu','fig','b','equation','equations','biological','biology','a','b','c','d','e','f','g','h','i','j',\
               'k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','use', 'uses','year','figure','studied','per',\
               'see','however','new','utc','http','https','et','al','one','two','study','studying','studies', 'also','used',\
               'thus','\u03c3','\u03c9','aug','thu','would','many','found','downloaded','press','university','within','de',\
               'could','should','results','result','effect','effects','total','given','using','table','likely','therefore',\
               'however','important','pp','among','ro','since','significant','first','\uxe4','must','well','levin','section',\
               'much','particular','second','first','show','based','part','biol','either','society']
common_words = ['host','hosts','population','populations','transmission','model','models','disease','pathogen','infectious', 'infection',\
               'infected']
words = [w for w in words if not w in stop_words]
words = [w for w in words if not w in ignore_words]
words = [w for w in words if not w in common_words]
#words = [w for w in words if not w in moretoignore]
words = [w for w in words if len(w) > 3]
lemma = WordNetLemmatizer()
#normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
words = [lemma.lemmatize(w) for w in words]

words_to_count = (word for word in words)
c = Counter(words_to_count)

In [4]:
mclist = c.most_common(500)
mclist = np.array(mclist)
mclist = mclist[:,0]
mclist = [x.encode('UTF8') for x in mclist]

In [5]:
mclist

['parasite',
 'rate',
 'virus',
 'number',
 'specie',
 'density',
 'human',
 'disease',
 'time',
 'control',
 'mortality',
 'individual',
 'size',
 'dynamic',
 'wildlife',
 'fish',
 'virulence',
 'parameter',
 'case',
 'change',
 'equilibrium',
 'level',
 'area',
 'evolution',
 'increase',
 'data',
 'system',
 'epidemic',
 'susceptible',
 'mean',
 'rabbit',
 'period',
 'animal',
 'contact',
 'value',
 'ecology',
 'condition',
 'interaction',
 'factor',
 'forest',
 'high',
 'analysis',
 'different',
 'conservation',
 'pathogen',
 'natural',
 'example',
 'experiment',
 'strain',
 'day',
 'higher',
 'growth',
 'female',
 'proportion',
 'probability',
 'variable',
 'school',
 'science',
 'response',
 'infection',
 'health',
 'reproduction',
 'solution',
 'resistance',
 'primate',
 'survival',
 'pattern',
 'death',
 'class',
 'type',
 'plant',
 'possum',
 'nature',
 'range',
 'average',
 'distribution',
 'although',
 'increased',
 'pool',
 'group',
 'impact',
 'research',
 'relationship',
 

In [6]:
#Now, build the corpus (This cell only needed to be called once to make the .txt files)
#filenames = open('../Papers/00/pdflist.txt').read().splitlines()
#for f in filenames:
 #   print f
  #  !pdftotext ../Papers/00/$f 

In [7]:
docslist = []
filenames = open('../Papers/00/textlist.txt').read().splitlines()
for f in filenames:
    txtfile = open('../Papers/00/'+f).read()
    docslist.append(txtfile)

In [8]:
wordlist = []
for raw in docslist:
    tokens = word_tokenize(raw.decode('utf-8'))
    tokens = [w.lower() for w in tokens]
    words = [word for word in tokens if word.isalpha()]
    words = [w for w in words if w in mclist]
    
    wordlist.append(words)

In [9]:
print len(wordlist)

64


In [10]:
dictionary = corpora.Dictionary(wordlist)
corpus = [dictionary.doc2bow(text) for text in wordlist]

In [17]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=20)

In [18]:
print(ldamodel.print_topics(num_topics=8, num_words=4))

[(0, u'0.044*"virus" + 0.029*"disease" + 0.026*"wildlife" + 0.021*"human"'), (1, u'0.031*"parasite" + 0.027*"daphnia" + 0.017*"disgust" + 0.016*"density"'), (2, u'0.033*"parasite" + 0.030*"fish" + 0.021*"infection" + 0.020*"size"'), (3, u'0.029*"rate" + 0.023*"infection" + 0.023*"disease" + 0.022*"number"'), (4, u'0.029*"parasite" + 0.029*"virulence" + 0.028*"disease" + 0.023*"infection"')]


In [19]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

In [ ]:
#Hmm. It seems the scikit-learn can do this, as well as NMF, which I think is actually more useful.
#https://nlpforhackers.io/topic-modeling/